-----
# PART-1

-----
### 1. Explore the Datasets

Load the saved Datasets

In [0]:
# Define the path of the saved Datasets
GREEN_DST = f"/Volumes/workspace/bde/assignment2/green"
YELLOW_DST = f"/Volumes/workspace/bde/assignment2/yellow"



In [0]:
# Directly read the parquet files without downloading again
green_df = spark.read.parquet(GREEN_DST)
yellow_df = spark.read.parquet(YELLOW_DST)

Chaeck the counts

In [0]:
# Check the count of rows in each DataFrame
green_count  = green_df.count()
yellow_count = yellow_df.count()
total_count  = green_count + yellow_count

print(green_count)
print(yellow_count)
print(total_count) #Should be equal to 991467464


83484688
907982776
991467464


View the first 5 rows of Datasets

In [0]:
# Display the first 5 rows of the green DataFrame
display(green_df.limit(5))

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
2,2014-01-01T00:17:26.000Z,2014-01-01T00:37:11.000Z,N,1.0,17,225,1.0,2.28,13.5,0.5,0.5,0.0,0.0,null,null,14.5,2.0,null,null
1,2014-01-01T00:29:12.000Z,2014-01-01T00:37:43.000Z,N,1.0,127,241,1.0,2.1,9.0,0.0,0.5,0.0,0.0,null,null,9.5,2.0,null,null
2,2014-01-01T00:31:35.000Z,2014-01-01T00:44:09.000Z,N,1.0,166,243,1.0,4.72,15.5,0.5,0.5,4.0,0.0,null,null,20.5,1.0,null,null
2,2014-01-01T00:07:01.000Z,2014-01-01T00:21:54.000Z,N,1.0,7,157,1.0,2.88,13.0,0.5,0.5,2.88,0.0,null,null,16.88,1.0,null,null
2,2014-01-01T00:26:43.000Z,2014-01-01T00:37:17.000Z,N,1.0,83,197,2.0,3.8,13.0,0.5,0.5,0.0,0.0,null,null,14.0,2.0,null,null


In [0]:
# Display the first 5 rows of the yellow DataFrame
display(yellow_df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2015-01-01T00:11:33.000Z,2015-01-01T00:16:48.000Z,1.0,1.0,1.0,N,41,166,1,5.7,0.5,0.5,1.4,0.0,0.0,8.4,null,null
1,2015-01-01T00:18:24.000Z,2015-01-01T00:24:20.000Z,1.0,0.9,1.0,N,166,238,3,6.0,0.5,0.5,0.0,0.0,0.0,7.3,null,null
1,2015-01-01T00:26:19.000Z,2015-01-01T00:41:06.000Z,1.0,3.5,1.0,N,238,162,1,13.2,0.5,0.5,2.9,0.0,0.0,17.4,null,null
1,2015-01-01T00:45:26.000Z,2015-01-01T00:53:20.000Z,1.0,2.1,1.0,N,162,263,1,8.2,0.5,0.5,2.37,0.0,0.0,11.87,null,null
1,2015-01-01T00:59:21.000Z,2015-01-01T01:05:24.000Z,1.0,1.0,1.0,N,236,141,3,6.0,0.5,0.5,0.0,0.0,0.0,7.3,null,null


View the Schemas of the Datasets

In [0]:
# Check the schema of the DataFrame
green_df.printSchema()


root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [0]:
# Check the schema of the DataFrame
yellow_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



Create a temp view for each DataFrame to clean the data

In [0]:
# Create a temp view for each DataFrame 
green_df.createOrReplaceTempView("green_taxi")
yellow_df.createOrReplaceTempView("yellow_taxi")


------
### 2. Clean the Datasets

#### a. Trips finishing before the starting time

In [0]:
%sql
-- Count the number of records with invalid start time
SELECT COUNT(*) AS invalid_start_time_count
FROM green_taxi
WHERE lpep_dropoff_datetime <= lpep_pickup_datetime;

invalid_start_time_count
68318


In [0]:
%sql
-- Count the number of records with invalid start time
SELECT COUNT(*) AS invalid_start_time_count
FROM yellow_taxi
WHERE tpep_dropoff_datetime <= tpep_pickup_datetime;

invalid_start_time_count
1108986


In [0]:
%sql
-- Create a new table by dropping the invalid records
CREATE OR REPLACE TABLE green_taxi_clean_start AS
SELECT *
FROM green_taxi
WHERE lpep_dropoff_datetime > lpep_pickup_datetime;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create a new table by dropping the invalid records
CREATE OR REPLACE TABLE yellow_taxi_clean_start AS
SELECT *
FROM yellow_taxi
WHERE tpep_dropoff_datetime > tpep_pickup_datetime;


num_affected_rows,num_inserted_rows


-----
#### b. Trips with negative speed

Calculate the time taken for the trip and save it as trip_time

In [0]:
%sql
-- Calculate the time by reducing the dropoff time from the pickup time and convert it in hours and save it as trip_time
CREATE OR REPLACE TEMP VIEW green_taxi_with_dist AS
SELECT *,((UNIX_TIMESTAMP(lpep_dropoff_datetime) - UNIX_TIMESTAMP(lpep_pickup_datetime)) / 3600.0)AS trip_time
FROM green_taxi_clean_start;

In [0]:
%sql
-- Calculate the time by reducing the dropoff time from the pickup time and convert it in hours and save it as trip_time
CREATE OR REPLACE TEMP VIEW yellow_taxi_with_dist AS
SELECT *,((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600.0)AS trip_time
FROM yellow_taxi_clean_start;

Calculate the speed of the trip and save it as speed_mph

In [0]:
%sql
-- Calculate the speed using formula(speed = distance/time) and save it as speed_mph
CREATE OR REPLACE TEMP VIEW green_taxi_with_speed AS
SELECT *,(trip_distance / trip_time )AS speed_mph
FROM green_taxi_with_dist;


In [0]:
%sql
-- Calculate the speed using formula(speed = distance/time) and save it as speed_mph
CREATE OR REPLACE TEMP VIEW yellow_taxi_with_speed AS
SELECT *,(trip_distance / trip_time )AS speed_mph
FROM yellow_taxi_with_dist;


In [0]:
%sql
-- Count the number of records with negative speed
SELECT Count(*) AS negative_speed_count
FROM green_taxi_with_speed 
WHERE speed_mph <= 0;


negative_speed_count
1300007


In [0]:
%sql
-- Count the number of records with negative speed
SELECT Count(*) AS negative_speed_count
FROM yellow_taxi_with_speed 
WHERE speed_mph <= 0;


negative_speed_count
6699144


In [0]:
%sql
-- Create table with valid speed records
CREATE OR REPLACE TABLE green_taxi_clean_speed AS
SELECT *
FROM green_taxi_with_speed
WHERE speed_mph > 0;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create table with valid speed records
CREATE OR REPLACE TABLE yellow_taxi_clean_speed  AS
SELECT *
FROM yellow_taxi_with_speed
WHERE speed_mph > 0;


num_affected_rows,num_inserted_rows


-----
#### c. Trips with very high speed (outside of NYC speed limit )

In [0]:
%sql
-- According to NYC 30MPH is the speed limit we will keep a threshold of 35MPH for our analysis
-- Count the number of records with speed more than 35MPH
SELECT Count(*) AS over_speed_count
FROM green_taxi_clean_speed
WHERE speed_mph >35; 

over_speed_count
700811


In [0]:
%sql
-- According to NYC 30MPH is the speed limit we will keep a threshold of 35MPH for our analysis
-- Count the number of records with speed more than 35MPH
SELECT Count(*) AS over_speed_count
FROM yellow_taxi_clean_speed
WHERE speed_mph >35;

over_speed_count
9535125


In [0]:
%sql
-- Create a table with legal speed records
CREATE OR REPLACE TABLE green_taxi_legal_speed AS
SELECT *
FROM green_taxi_clean_speed
WHERE speed_mph < 35;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create a table with legal speed records
CREATE OR REPLACE TABLE yellow_taxi_legal_speed AS
SELECT *
FROM yellow_taxi_clean_speed
WHERE speed_mph <35;


num_affected_rows,num_inserted_rows


-----
#### d. Trips that are travelling too short or too long (duration wise)

In [0]:
%sql
-- Find the Min and Max trip_time
SELECT MAX(trip_time) as max_trip_time,
MIN(trip_time) as min_trip_time
FROM green_taxi_legal_speed;

max_trip_time,min_trip_time
139.376667,0.000556


In [0]:
%sql
-- Find the Min and Max trip_time
SELECT MAX(trip_time) as max_trip_time,
MIN(trip_time) as min_trip_time
FROM yellow_taxi_legal_speed;

max_trip_time,min_trip_time
2089552.680556,0.000556


In [0]:
%sql
-- Count the number of records with trip_time more than 6 hours or less than 1 minute
SELECT Count(*) AS out_of_range_count
FROM green_taxi_legal_speed
WHERE trip_time > 6 OR trip_time < (1.0/60.0) ;

out_of_range_count
788061


In [0]:
%sql
-- Count the number of records with trip_time more than 6 hours or less than 1 minute
SELECT Count(*) AS out_of_range_count
FROM yellow_taxi_legal_speed
WHERE trip_time > 6 OR trip_time < (1.0/60.0) ;

out_of_range_count
3402686


In [0]:
%sql
-- Create a table where trip_time is between 1 minute and 6 hours
CREATE OR REPLACE TABLE green_taxi_clean1 AS
SELECT *
FROM green_taxi_legal_speed
WHERE trip_time > (1.0/60.0) AND trip_time < 6;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create a table where trip_time is between 1 minute and 6 hours
CREATE OR REPLACE TABLE yellow_taxi_clean1 AS
SELECT *
FROM yellow_taxi_legal_speed
WHERE trip_time > (1.0/60.0) AND trip_time < 6;

num_affected_rows,num_inserted_rows


-----
#### e. Trips that are travelling too short or too long (distance wise)

In [0]:
%sql
-- Find the Min and Max trip_distance
SELECT MAX(trip_distance) as max_trip_distance,
MIN(trip_distance) as min_trip_distance
FROM green_taxi_clean1;

max_trip_distance,min_trip_distance
188.65,0.01


In [0]:
%sql
-- Find the Min and Max trip_distance
SELECT MAX(trip_distance) as max_trip_distance,
MIN(trip_distance) as min_trip_distance
FROM yellow_taxi_clean1;

max_trip_distance,min_trip_distance
198.12,0.01


In [0]:
%sql
-- Count the number of records with trip_distance less than 0.1 mile or more than 100 miles
SELECT COUNT(*) AS out_of_range_count
FROM green_taxi_clean1
WHERE trip_distance < 0.1
   OR trip_distance > 100;


out_of_range_count
109298


In [0]:
%sql
-- Count the number of records with trip_distance less than 0.1 mile or more than 100 miles
SELECT COUNT(*) AS out_of_range_count
FROM yellow_taxi_clean1
WHERE trip_distance < 0.1
   OR trip_distance > 100;


out_of_range_count
564828


In [0]:
%sql
-- Create a table where trip_distance is between 0.1 mile and 100 miles
CREATE OR REPLACE TABLE green_taxi_clean2 AS
SELECT *
FROM green_taxi_clean1
WHERE trip_distance > 0.1 AND trip_distance < 100;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create a table where trip_distance is between 0.1 mile and 100 miles
CREATE OR REPLACE TABLE yellow_taxi_clean2 AS
SELECT *
FROM yellow_taxi_clean1
WHERE trip_distance > 0.1 AND trip_distance < 100;


num_affected_rows,num_inserted_rows


-----
#### f. Trips with Invalid Payment Types

In [0]:
%sql
-- We have found the valid payment_types from the data dictionary
-- Count the number of records with invalid payment_type
SELECT COUNT(*) as invalid_payment_type
FROM green_taxi_clean2
WHERE payment_type NOT IN (0, 1, 2, 3, 4, 5, 6);


invalid_payment_type
0


In [0]:
%sql
-- We have found the valid payment_types from the data dictionary
-- Count the number of records with invalid payment_type
SELECT COUNT(*) as invalid_payment_type
FROM yellow_taxi_clean2
WHERE payment_type NOT IN (0, 1, 2, 3, 4, 5, 6);


invalid_payment_type
0


-----
#### g. Trips with invalid RatecodeID

In [0]:
%sql
-- We have found the valid ratecodeID from the data dictionary
-- Count the number of records with invalid ratecodeID
SELECT COUNT(*) as invalid_ratecodeID
FROM green_taxi_clean2
WHERE RatecodeID NOT IN (1, 2, 3, 4, 5, 6, 99);


invalid_ratecodeID
0


In [0]:
%sql
-- We have found the valid ratecodeID from the data dictionary
-- Count the number of records with invalid ratecodeID
SELECT COUNT(*) as invalid_ratecodeID
FROM yellow_taxi_clean2
WHERE RatecodeID NOT IN (1, 2, 3, 4, 5, 6, 99);


invalid_ratecodeID
0


-----
#### h. Trips with Invalid VendorID

In [0]:
%sql
-- We have found the valid VendorID from the data dictionary
-- Count the number of records with invalid VendorID
SELECT COUNT(*) as invalid_vendorID
FROM green_taxi_clean2
WHERE VendorID NOT IN (1, 2, 6);


invalid_vendorID
45


In [0]:
%sql
-- We have found the valid VendorID from the data dictionary
-- Count the number of records with invalid VendorID
SELECT COUNT(*) as invalid_vendorID
FROM yellow_taxi_clean2
WHERE VendorID NOT IN (1, 2, 6, 7);


invalid_vendorID
753268


In [0]:
%sql
-- create a table with valid VendorID
CREATE OR REPLACE TABLE green_taxi_vendor AS
SELECT *
FROM green_taxi_clean2
WHERE VendorID IN (1,2,6);

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- create a table with valid VendorID
CREATE OR REPLACE TABLE yellow_taxi_vendor AS
SELECT *
FROM yellow_taxi_clean2
WHERE VendorID IN (1,2,6,7);

num_affected_rows,num_inserted_rows


-----
#### i. Handle Null values

**Green Taxi**

In [0]:
%sql
-- Count the number of records with null values in each column
SELECT
    SUM(CASE WHEN VendorID IS NULL THEN 1 ELSE 0 END) AS VendorID_nulls,
    SUM(CASE WHEN lpep_pickup_datetime IS NULL THEN 1 ELSE 0 END) AS pickup_nulls,
    SUM(CASE WHEN lpep_dropoff_datetime IS NULL THEN 1 ELSE 0 END) AS dropoff_nulls,
    SUM(CASE WHEN store_and_fwd_flag IS NULL THEN 1 ELSE 0 END) AS store_fwd_nulls,
    SUM(CASE WHEN RatecodeID IS NULL THEN 1 ELSE 0 END) AS ratecode_nulls,
    SUM(CASE WHEN PULocationID IS NULL THEN 1 ELSE 0 END) AS pu_location_nulls,
    SUM(CASE WHEN DOLocationID IS NULL THEN 1 ELSE 0 END) AS do_location_nulls,
    SUM(CASE WHEN passenger_count IS NULL THEN 1 ELSE 0 END) AS passenger_nulls,
    SUM(CASE WHEN trip_distance IS NULL THEN 1 ELSE 0 END) AS distance_nulls,
    SUM(CASE WHEN fare_amount IS NULL THEN 1 ELSE 0 END) AS fare_nulls,
    SUM(CASE WHEN extra IS NULL THEN 1 ELSE 0 END) AS extra_nulls,
    SUM(CASE WHEN mta_tax IS NULL THEN 1 ELSE 0 END) AS mta_tax_nulls,
    SUM(CASE WHEN tip_amount IS NULL THEN 1 ELSE 0 END) AS tip_nulls,
    SUM(CASE WHEN tolls_amount IS NULL THEN 1 ELSE 0 END) AS tolls_nulls,
    SUM(CASE WHEN ehail_fee IS NULL THEN 1 ELSE 0 END) AS ehail_nulls,
    SUM(CASE WHEN improvement_surcharge IS NULL THEN 1 ELSE 0 END) AS surcharge_nulls,
    SUM(CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END) AS total_nulls,
    SUM(CASE WHEN payment_type IS NULL THEN 1 ELSE 0 END) AS payment_nulls,
    SUM(CASE WHEN trip_type IS NULL THEN 1 ELSE 0 END) AS trip_type_nulls,
    SUM(CASE WHEN congestion_surcharge IS NULL THEN 1 ELSE 0 END) AS congestion_nulls,
    SUM(CASE WHEN trip_time IS NULL THEN 1 ELSE 0 END) AS trip_time_nulls,
    SUM(CASE WHEN speed_mph IS NULL THEN 1 ELSE 0 END) AS speed_nulls
FROM green_taxi_vendor;


VendorID_nulls,pickup_nulls,dropoff_nulls,store_fwd_nulls,ratecode_nulls,pu_location_nulls,do_location_nulls,passenger_nulls,distance_nulls,fare_nulls,extra_nulls,mta_tax_nulls,tip_nulls,tolls_nulls,ehail_nulls,surcharge_nulls,total_nulls,payment_nulls,trip_type_nulls,congestion_nulls,trip_time_nulls,speed_nulls
0,0,0,1796234,1796234,0,0,1796234,0,0,0,0,0,0,80459695,13693765,0,1796234,3506577,71931344,0,0


In [0]:
%sql
-- create a table by dropping the trip_type column null values
CREATE OR REPLACE TABLE green_taxi_clean AS
SELECT *
FROM green_taxi_vendor
WHERE trip_type IS NOT NULL;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Count the number of records with null values in each column again
SELECT
    SUM(CASE WHEN VendorID IS NULL THEN 1 ELSE 0 END) AS VendorID_nulls,
    SUM(CASE WHEN lpep_pickup_datetime IS NULL THEN 1 ELSE 0 END) AS pickup_nulls,
    SUM(CASE WHEN lpep_dropoff_datetime IS NULL THEN 1 ELSE 0 END) AS dropoff_nulls,
    SUM(CASE WHEN store_and_fwd_flag IS NULL THEN 1 ELSE 0 END) AS store_fwd_nulls,
    SUM(CASE WHEN RatecodeID IS NULL THEN 1 ELSE 0 END) AS ratecode_nulls,
    SUM(CASE WHEN PULocationID IS NULL THEN 1 ELSE 0 END) AS pu_location_nulls,
    SUM(CASE WHEN DOLocationID IS NULL THEN 1 ELSE 0 END) AS do_location_nulls,
    SUM(CASE WHEN passenger_count IS NULL THEN 1 ELSE 0 END) AS passenger_nulls,
    SUM(CASE WHEN trip_distance IS NULL THEN 1 ELSE 0 END) AS distance_nulls,
    SUM(CASE WHEN fare_amount IS NULL THEN 1 ELSE 0 END) AS fare_nulls,
    SUM(CASE WHEN extra IS NULL THEN 1 ELSE 0 END) AS extra_nulls,
    SUM(CASE WHEN mta_tax IS NULL THEN 1 ELSE 0 END) AS mta_tax_nulls,
    SUM(CASE WHEN tip_amount IS NULL THEN 1 ELSE 0 END) AS tip_nulls,
    SUM(CASE WHEN tolls_amount IS NULL THEN 1 ELSE 0 END) AS tolls_nulls,
    SUM(CASE WHEN ehail_fee IS NULL THEN 1 ELSE 0 END) AS ehail_nulls,
    SUM(CASE WHEN improvement_surcharge IS NULL THEN 1 ELSE 0 END) AS surcharge_nulls,
    SUM(CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END) AS total_nulls,
    SUM(CASE WHEN payment_type IS NULL THEN 1 ELSE 0 END) AS payment_nulls,
    SUM(CASE WHEN trip_type IS NULL THEN 1 ELSE 0 END) AS trip_type_nulls,
    SUM(CASE WHEN congestion_surcharge IS NULL THEN 1 ELSE 0 END) AS congestion_nulls,
    SUM(CASE WHEN trip_time IS NULL THEN 1 ELSE 0 END) AS trip_time_nulls,
    SUM(CASE WHEN speed_mph IS NULL THEN 1 ELSE 0 END) AS speed_nulls
FROM green_taxi_clean;


VendorID_nulls,pickup_nulls,dropoff_nulls,store_fwd_nulls,ratecode_nulls,pu_location_nulls,do_location_nulls,passenger_nulls,distance_nulls,fare_nulls,extra_nulls,mta_tax_nulls,tip_nulls,tolls_nulls,ehail_nulls,surcharge_nulls,total_nulls,payment_nulls,trip_type_nulls,congestion_nulls,trip_time_nulls,speed_nulls
0,0,0,0,0,0,0,0,0,0,0,0,0,0,76956054,11986451,0,0,0,68426764,0,0


Yellow Taxi

In [0]:
%sql
-- Count the number of records with null values in each column
SELECT
    SUM(CASE WHEN VendorID IS NULL THEN 1 ELSE 0 END) AS VendorID_nulls,
    SUM(CASE WHEN tpep_pickup_datetime IS NULL THEN 1 ELSE 0 END) AS pickup_nulls,
    SUM(CASE WHEN tpep_dropoff_datetime IS NULL THEN 1 ELSE 0 END) AS dropoff_nulls,
    SUM(CASE WHEN passenger_count IS NULL THEN 1 ELSE 0 END) AS passenger_nulls,
    SUM(CASE WHEN trip_distance IS NULL THEN 1 ELSE 0 END) AS distance_nulls,
    SUM(CASE WHEN RatecodeID IS NULL THEN 1 ELSE 0 END) AS ratecode_nulls,
    SUM(CASE WHEN store_and_fwd_flag IS NULL THEN 1 ELSE 0 END) AS store_fwd_nulls,
    SUM(CASE WHEN PULocationID IS NULL THEN 1 ELSE 0 END) AS pu_location_nulls,
    SUM(CASE WHEN DOLocationID IS NULL THEN 1 ELSE 0 END) AS do_location_nulls,
    SUM(CASE WHEN payment_type IS NULL THEN 1 ELSE 0 END) AS payment_nulls,
    SUM(CASE WHEN fare_amount IS NULL THEN 1 ELSE 0 END) AS fare_nulls,
    SUM(CASE WHEN extra IS NULL THEN 1 ELSE 0 END) AS extra_nulls,
    SUM(CASE WHEN mta_tax IS NULL THEN 1 ELSE 0 END) AS mta_tax_nulls,
    SUM(CASE WHEN tip_amount IS NULL THEN 1 ELSE 0 END) AS tip_nulls,
    SUM(CASE WHEN tolls_amount IS NULL THEN 1 ELSE 0 END) AS tolls_nulls,
    SUM(CASE WHEN improvement_surcharge IS NULL THEN 1 ELSE 0 END) AS surcharge_nulls,
    SUM(CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END) AS total_nulls,
    SUM(CASE WHEN congestion_surcharge IS NULL THEN 1 ELSE 0 END) AS congestion_nulls,
    SUM(CASE WHEN airport_fee IS NULL THEN 1 ELSE 0 END) AS airport_fee_nulls,
    SUM(CASE WHEN trip_time IS NULL THEN 1 ELSE 0 END) AS trip_time_nulls,
    SUM(CASE WHEN speed_mph IS NULL THEN 1 ELSE 0 END) AS speed_nulls
FROM yellow_taxi_vendor;


VendorID_nulls,pickup_nulls,dropoff_nulls,passenger_nulls,distance_nulls,ratecode_nulls,store_fwd_nulls,pu_location_nulls,do_location_nulls,payment_nulls,fare_nulls,extra_nulls,mta_tax_nulls,tip_nulls,tolls_nulls,surcharge_nulls,total_nulls,congestion_nulls,airport_fee_nulls,trip_time_nulls,speed_nulls
0,0,0,8571656,0,8571656,57426682,0,0,0,0,0,0,0,0,52404914,0,647261449,820411767,0,0


In [0]:
%sql
-- create a table by dropping the RatecodeID column null values
CREATE OR REPLACE TABLE yellow_taxi_clean AS
SELECT *
FROM yellow_taxi_vendor
WHERE RatecodeID IS NOT NULL;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Count the number of records with null values in each column again
SELECT
    SUM(CASE WHEN VendorID IS NULL THEN 1 ELSE 0 END) AS VendorID_nulls,
    SUM(CASE WHEN tpep_pickup_datetime IS NULL THEN 1 ELSE 0 END) AS pickup_nulls,
    SUM(CASE WHEN tpep_dropoff_datetime IS NULL THEN 1 ELSE 0 END) AS dropoff_nulls,
    SUM(CASE WHEN passenger_count IS NULL THEN 1 ELSE 0 END) AS passenger_nulls,
    SUM(CASE WHEN trip_distance IS NULL THEN 1 ELSE 0 END) AS distance_nulls,
    SUM(CASE WHEN RatecodeID IS NULL THEN 1 ELSE 0 END) AS ratecode_nulls,
    SUM(CASE WHEN store_and_fwd_flag IS NULL THEN 1 ELSE 0 END) AS store_fwd_nulls,
    SUM(CASE WHEN PULocationID IS NULL THEN 1 ELSE 0 END) AS pu_location_nulls,
    SUM(CASE WHEN DOLocationID IS NULL THEN 1 ELSE 0 END) AS do_location_nulls,
    SUM(CASE WHEN payment_type IS NULL THEN 1 ELSE 0 END) AS payment_nulls,
    SUM(CASE WHEN fare_amount IS NULL THEN 1 ELSE 0 END) AS fare_nulls,
    SUM(CASE WHEN extra IS NULL THEN 1 ELSE 0 END) AS extra_nulls,
    SUM(CASE WHEN mta_tax IS NULL THEN 1 ELSE 0 END) AS mta_tax_nulls,
    SUM(CASE WHEN tip_amount IS NULL THEN 1 ELSE 0 END) AS tip_nulls,
    SUM(CASE WHEN tolls_amount IS NULL THEN 1 ELSE 0 END) AS tolls_nulls,
    SUM(CASE WHEN improvement_surcharge IS NULL THEN 1 ELSE 0 END) AS surcharge_nulls,
    SUM(CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END) AS total_nulls,
    SUM(CASE WHEN congestion_surcharge IS NULL THEN 1 ELSE 0 END) AS congestion_nulls,
    SUM(CASE WHEN airport_fee IS NULL THEN 1 ELSE 0 END) AS airport_fee_nulls,
    SUM(CASE WHEN trip_time IS NULL THEN 1 ELSE 0 END) AS trip_time_nulls,
    SUM(CASE WHEN speed_mph IS NULL THEN 1 ELSE 0 END) AS speed_nulls
FROM yellow_taxi_clean;


VendorID_nulls,pickup_nulls,dropoff_nulls,passenger_nulls,distance_nulls,ratecode_nulls,store_fwd_nulls,pu_location_nulls,do_location_nulls,payment_nulls,fare_nulls,extra_nulls,mta_tax_nulls,tip_nulls,tolls_nulls,surcharge_nulls,total_nulls,congestion_nulls,airport_fee_nulls,trip_time_nulls,speed_nulls
0,0,0,0,0,0,48855026,0,0,0,0,0,0,0,0,52404914,0,638689793,811840111,0,0


------
#### j. Trips with Odd Passanger Count

In [0]:
%sql
-- count the records with passenger_count not between 1 and 6
SELECT COUNT(*) AS high_passenger_count
FROM green_taxi_clean
WHERE passenger_count NOT BETWEEN 1 AND 6;


high_passenger_count
48666


In [0]:
%sql
-- count the records with passenger_count not between 1 and 6
SELECT COUNT(*) AS high_passanger_count
FROM yellow_taxi_clean
WHERE passenger_count NOT BETWEEN 1 AND 6;

high_passanger_count
5374288


In [0]:
%sql
-- Create the table with passenger_count between 1 and 6 
CREATE OR REPLACE TABLE green_taxi_pass AS
SELECT *
FROM green_taxi_clean
WHERE passenger_count BETWEEN 1 AND 6;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create the table with passenger_count between 1 and 6
CREATE OR REPLACE TABLE yellow_taxi_pass AS
SELECT *
FROM yellow_taxi_clean
WHERE passenger_count BETWEEN 1 AND 6;

num_affected_rows,num_inserted_rows


### k. Trips where pickup/dropoff datetime is outside the range

In [0]:
%sql
-- count the number of records with out of range datetime
-- We found that the green taxis are added to the fleet only in 2013.
SELECT COUNT(*) AS outofrange_green
FROM green_taxi_pass
WHERE lpep_pickup_datetime < '2013-01-01'
   OR lpep_pickup_datetime > '2025-06-30'
   OR lpep_dropoff_datetime < '2013-01-01'
   OR lpep_dropoff_datetime > '2025-06-30';

outofrange_green
544


In [0]:
%sql
-- count the number of records with out of range datetime
-- We found that the taxi trip datas are collected from 2009.
SELECT COUNT(*) AS outofrange_yellow
FROM yellow_taxi_pass
WHERE tpep_pickup_datetime < '2009-01-01'
   OR tpep_pickup_datetime > '2025-06-30'
   OR tpep_dropoff_datetime < '2009-01-01'
   OR tpep_dropoff_datetime > '2025-06-30';

outofrange_yellow
853


In [0]:
%sql
-- Create the table with correct datetime range
CREATE OR REPLACE TABLE green_taxi_range AS
SELECT *
FROM green_taxi_pass
WHERE lpep_pickup_datetime >= '2013-01-01'
  AND lpep_pickup_datetime <= '2025-06-30'
  AND lpep_dropoff_datetime >= '2013-01-01'
  AND lpep_dropoff_datetime <= '2025-06-30';

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create the table with correct datetime range
CREATE OR REPLACE TABLE yellow_taxi_range AS
SELECT *
FROM yellow_taxi_pass
WHERE tpep_pickup_datetime >= '2009-01-01'
  AND tpep_pickup_datetime <= '2025-06-30'
  AND tpep_dropoff_datetime >= '2009-01-01'
  AND tpep_dropoff_datetime <= '2025-06-30';

num_affected_rows,num_inserted_rows


----
## l. Handle the Negative Values

In [0]:
%sql
-- Count the number of records with negative values in each column
SELECT
    SUM(CASE WHEN VendorID < 0 THEN 1 ELSE 0 END) AS VendorID_negatives,
    SUM(CASE WHEN RatecodeID < 0 THEN 1 ELSE 0 END) AS ratecode_negatives,
    SUM(CASE WHEN PULocationID < 0 THEN 1 ELSE 0 END) AS pu_location_negatives,
    SUM(CASE WHEN DOLocationID < 0 THEN 1 ELSE 0 END) AS do_location_negatives,
    SUM(CASE WHEN passenger_count < 0 THEN 1 ELSE 0 END) AS passenger_negatives,
    SUM(CASE WHEN trip_distance < 0 THEN 1 ELSE 0 END) AS distance_negatives,
    SUM(CASE WHEN fare_amount < 0 THEN 1 ELSE 0 END) AS fare_negatives,
    SUM(CASE WHEN extra < 0 THEN 1 ELSE 0 END) AS extra_negatives,
    SUM(CASE WHEN mta_tax < 0 THEN 1 ELSE 0 END) AS mta_tax_negatives,
    SUM(CASE WHEN tip_amount < 0 THEN 1 ELSE 0 END) AS tip_negatives,
    SUM(CASE WHEN tolls_amount < 0 THEN 1 ELSE 0 END) AS tolls_negatives,
    SUM(CASE WHEN ehail_fee < 0 THEN 1 ELSE 0 END) AS ehail_negatives,
    SUM(CASE WHEN improvement_surcharge < 0 THEN 1 ELSE 0 END) AS surcharge_negatives,
    SUM(CASE WHEN total_amount < 0 THEN 1 ELSE 0 END) AS total_negatives,
    SUM(CASE WHEN payment_type < 0 THEN 1 ELSE 0 END) AS payment_negatives,
    SUM(CASE WHEN trip_type < 0 THEN 1 ELSE 0 END) AS trip_type_negatives,
    SUM(CASE WHEN congestion_surcharge < 0 THEN 1 ELSE 0 END) AS congestion_negatives,
    SUM(CASE WHEN trip_time < 0 THEN 1 ELSE 0 END) AS trip_time_negatives,
    SUM(CASE WHEN speed_mph < 0 THEN 1 ELSE 0 END) AS speed_negatives
FROM green_taxi_range;


VendorID_negatives,ratecode_negatives,pu_location_negatives,do_location_negatives,passenger_negatives,distance_negatives,fare_negatives,extra_negatives,mta_tax_negatives,tip_negatives,tolls_negatives,ehail_negatives,surcharge_negatives,total_negatives,payment_negatives,trip_type_negatives,congestion_negatives,trip_time_negatives,speed_negatives
0,0,0,0,0,0,75552,42545,73987,324,28,0,67219,75644,0,0,187,0,0


In [0]:
%sql
-- Create the table with negative values removed
CREATE OR REPLACE TABLE green_taxi_positive AS
SELECT *
FROM green_taxi_range
WHERE total_amount >= 0
AND extra >= 0
AND tip_amount >= 0;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Count the number of records with negative values in each column again to ensure everything is removed
SELECT
    SUM(CASE WHEN VendorID < 0 THEN 1 ELSE 0 END) AS VendorID_negatives,
    SUM(CASE WHEN RatecodeID < 0 THEN 1 ELSE 0 END) AS ratecode_negatives,
    SUM(CASE WHEN PULocationID < 0 THEN 1 ELSE 0 END) AS pu_location_negatives,
    SUM(CASE WHEN DOLocationID < 0 THEN 1 ELSE 0 END) AS do_location_negatives,
    SUM(CASE WHEN passenger_count < 0 THEN 1 ELSE 0 END) AS passenger_negatives,
    SUM(CASE WHEN trip_distance < 0 THEN 1 ELSE 0 END) AS distance_negatives,
    SUM(CASE WHEN fare_amount < 0 THEN 1 ELSE 0 END) AS fare_negatives,
    SUM(CASE WHEN extra < 0 THEN 1 ELSE 0 END) AS extra_negatives,
    SUM(CASE WHEN mta_tax < 0 THEN 1 ELSE 0 END) AS mta_tax_negatives,
    SUM(CASE WHEN tip_amount < 0 THEN 1 ELSE 0 END) AS tip_negatives,
    SUM(CASE WHEN tolls_amount < 0 THEN 1 ELSE 0 END) AS tolls_negatives,
    SUM(CASE WHEN ehail_fee < 0 THEN 1 ELSE 0 END) AS ehail_negatives,
    SUM(CASE WHEN improvement_surcharge < 0 THEN 1 ELSE 0 END) AS surcharge_negatives,
    SUM(CASE WHEN total_amount < 0 THEN 1 ELSE 0 END) AS total_negatives,
    SUM(CASE WHEN payment_type < 0 THEN 1 ELSE 0 END) AS payment_negatives,
    SUM(CASE WHEN trip_type < 0 THEN 1 ELSE 0 END) AS trip_type_negatives,
    SUM(CASE WHEN congestion_surcharge < 0 THEN 1 ELSE 0 END) AS congestion_negatives,
    SUM(CASE WHEN trip_time < 0 THEN 1 ELSE 0 END) AS trip_time_negatives,
    SUM(CASE WHEN speed_mph < 0 THEN 1 ELSE 0 END) AS speed_negatives
FROM green_taxi_positive;


VendorID_negatives,ratecode_negatives,pu_location_negatives,do_location_negatives,passenger_negatives,distance_negatives,fare_negatives,extra_negatives,mta_tax_negatives,tip_negatives,tolls_negatives,ehail_negatives,surcharge_negatives,total_negatives,payment_negatives,trip_type_negatives,congestion_negatives,trip_time_negatives,speed_negatives
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
%sql
-- Count the number of records with negative values in each column
SELECT
    SUM(CASE WHEN VendorID < 0 THEN 1 ELSE 0 END) AS VendorID_negatives,
    SUM(CASE WHEN passenger_count < 0 THEN 1 ELSE 0 END) AS passenger_negatives,
    SUM(CASE WHEN trip_distance < 0 THEN 1 ELSE 0 END) AS distance_negatives,
    SUM(CASE WHEN RatecodeID < 0 THEN 1 ELSE 0 END) AS ratecode_negatives,
    SUM(CASE WHEN PULocationID < 0 THEN 1 ELSE 0 END) AS pu_location_negatives,
    SUM(CASE WHEN DOLocationID < 0 THEN 1 ELSE 0 END) AS do_location_negatives,
    SUM(CASE WHEN payment_type < 0 THEN 1 ELSE 0 END) AS payment_negatives,
    SUM(CASE WHEN fare_amount < 0 THEN 1 ELSE 0 END) AS fare_negatives,
    SUM(CASE WHEN extra < 0 THEN 1 ELSE 0 END) AS extra_negatives,
    SUM(CASE WHEN mta_tax < 0 THEN 1 ELSE 0 END) AS mta_tax_negatives,
    SUM(CASE WHEN tip_amount < 0 THEN 1 ELSE 0 END) AS tip_negatives,
    SUM(CASE WHEN tolls_amount < 0 THEN 1 ELSE 0 END) AS tolls_negatives,
    SUM(CASE WHEN improvement_surcharge < 0 THEN 1 ELSE 0 END) AS surcharge_negatives,
    SUM(CASE WHEN total_amount < 0 THEN 1 ELSE 0 END) AS total_negatives,
    SUM(CASE WHEN congestion_surcharge < 0 THEN 1 ELSE 0 END) AS congestion_negatives,
    SUM(CASE WHEN airport_fee < 0 THEN 1 ELSE 0 END) AS airport_fee_negatives,
    SUM(CASE WHEN trip_time < 0 THEN 1 ELSE 0 END) AS trip_time_negatives,
    SUM(CASE WHEN speed_mph < 0 THEN 1 ELSE 0 END) AS speed_negatives
FROM yellow_taxi_range;


VendorID_negatives,passenger_negatives,distance_negatives,ratecode_negatives,pu_location_negatives,do_location_negatives,payment_negatives,fare_negatives,extra_negatives,mta_tax_negatives,tip_negatives,tolls_negatives,surcharge_negatives,total_negatives,congestion_negatives,airport_fee_negatives,trip_time_negatives,speed_negatives
0,0,0,0,0,0,0,1361119,695920,1346495,3360,80940,1351071,1365303,1095435,21923,0,0


In [0]:
%sql
-- Create the table with negative values removed
CREATE OR REPLACE TABLE yellow_taxi_positive AS
SELECT *
FROM yellow_taxi_range
WHERE total_amount >= 0
AND extra >= 0
AND fare_amount >= 0
AND tolls_amount >= 0
AND tip_amount >= 0;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Count the number of records with negative values in each column again to ensure everything is removed
SELECT
    SUM(CASE WHEN VendorID < 0 THEN 1 ELSE 0 END) AS VendorID_negatives,
    SUM(CASE WHEN passenger_count < 0 THEN 1 ELSE 0 END) AS passenger_negatives,
    SUM(CASE WHEN trip_distance < 0 THEN 1 ELSE 0 END) AS distance_negatives,
    SUM(CASE WHEN RatecodeID < 0 THEN 1 ELSE 0 END) AS ratecode_negatives,
    SUM(CASE WHEN PULocationID < 0 THEN 1 ELSE 0 END) AS pu_location_negatives,
    SUM(CASE WHEN DOLocationID < 0 THEN 1 ELSE 0 END) AS do_location_negatives,
    SUM(CASE WHEN payment_type < 0 THEN 1 ELSE 0 END) AS payment_negatives,
    SUM(CASE WHEN fare_amount < 0 THEN 1 ELSE 0 END) AS fare_negatives,
    SUM(CASE WHEN extra < 0 THEN 1 ELSE 0 END) AS extra_negatives,
    SUM(CASE WHEN mta_tax < 0 THEN 1 ELSE 0 END) AS mta_tax_negatives,
    SUM(CASE WHEN tip_amount < 0 THEN 1 ELSE 0 END) AS tip_negatives,
    SUM(CASE WHEN tolls_amount < 0 THEN 1 ELSE 0 END) AS tolls_negatives,
    SUM(CASE WHEN improvement_surcharge < 0 THEN 1 ELSE 0 END) AS surcharge_negatives,
    SUM(CASE WHEN total_amount < 0 THEN 1 ELSE 0 END) AS total_negatives,
    SUM(CASE WHEN congestion_surcharge < 0 THEN 1 ELSE 0 END) AS congestion_negatives,
    SUM(CASE WHEN airport_fee < 0 THEN 1 ELSE 0 END) AS airport_fee_negatives,
    SUM(CASE WHEN trip_time < 0 THEN 1 ELSE 0 END) AS trip_time_negatives,
    SUM(CASE WHEN speed_mph < 0 THEN 1 ELSE 0 END) AS speed_negatives
FROM yellow_taxi_positive;


VendorID_negatives,passenger_negatives,distance_negatives,ratecode_negatives,pu_location_negatives,do_location_negatives,payment_negatives,fare_negatives,extra_negatives,mta_tax_negatives,tip_negatives,tolls_negatives,surcharge_negatives,total_negatives,congestion_negatives,airport_fee_negatives,trip_time_negatives,speed_negatives
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


-----
### m. Trips Where Total_amount out of range (too high or zero)

In [0]:
%sql
-- Check the min, max, mean, median to set the threshold
select min(total_amount), max(total_amount), avg(total_amount), median(total_amount) from green_taxi_positive;

min(total_amount),max(total_amount),avg(total_amount),median(total_amount)
0.0,989970.39,14.931121023681856,11.6


In [0]:
%sql
-- Check the min, max, mean, median to set the threshold
select min(total_amount), max(total_amount), avg(total_amount), median(total_amount) from yellow_taxi_positive;

min(total_amount),max(total_amount),avg(total_amount),median(total_amount)
0.0,1.0000015E7,17.277931759451498,12.96


In [0]:
%sql
-- Count the number of trips where total amount > 100 and where total amount=0 but payment type != 3 (no charge)
SELECT COUNT(*) FROM green_taxi_positive
WHERE total_amount >100
OR (total_amount=0 AND payment_type!=3);

COUNT(*)
175357


In [0]:
%sql
-- Count the number of trips where total amount > 100 and where total amount=0 but payment type != 3 (no charge)
SELECT COUNT(*) FROM yellow_taxi_positive
WHERE total_amount >100
OR (total_amount=0 AND payment_type!=3);

COUNT(*)
1668121


In [0]:
%sql
-- Create a table with considerable total amounts
CREATE OR REPLACE TABLE green_taxi_final AS
SELECT * FROM green_taxi_positive
WHERE total_amount <=100
  AND (total_amount > 0 OR payment_type = 3); 
    

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Create a table with considerable total amounts
CREATE OR REPLACE TABLE yellow_taxi_final AS
SELECT * FROM yellow_taxi_positive
WHERE total_amount <=100
  AND (total_amount > 0 OR payment_type = 3); 
    

num_affected_rows,num_inserted_rows


-----
## 3. Count the total number of rows

In [0]:
%sql
-- Count the number of records in the final green table
SELECT COUNT(*) AS green_final_count
FROM green_taxi_final;

green_final_count
76655821


In [0]:
%sql
-- Count the number of records in the final yellow table
SELECT COUNT(*) AS yellow_final_count
FROM yellow_taxi_final;

yellow_final_count
867824328


-----
## 4. Combine the yellow and green taxi dataset

In [0]:
%sql
-- Standardize green taxi
CREATE OR REPLACE TEMP VIEW green_taxi_standard AS
SELECT
  VendorID,
  lpep_pickup_datetime AS pickup_datetime,
  lpep_dropoff_datetime AS dropoff_datetime,
  passenger_count,
  trip_distance,
  RatecodeID,
  store_and_fwd_flag,
  PULocationID,
  DOLocationID,
  payment_type,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  ehail_fee,
  improvement_surcharge,
  total_amount,
  trip_type,
  congestion_surcharge,
  trip_time,
  speed_mph,
  CAST(NULL AS DOUBLE) AS airport_fee,   -- not in green
  'green' AS taxi_colour                -- added constant column
FROM green_taxi_final;

In [0]:
%sql
-- Standardize yellow taxi
CREATE OR REPLACE TEMP VIEW yellow_taxi_standard AS
SELECT
  VendorID,
  tpep_pickup_datetime AS pickup_datetime,
  tpep_dropoff_datetime AS dropoff_datetime,
  passenger_count,
  trip_distance,
  RatecodeID,
  store_and_fwd_flag,
  PULocationID,
  DOLocationID,
  payment_type,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  CAST(NULL AS DOUBLE) AS ehail_fee,    -- not in yellow
  improvement_surcharge,
  total_amount,
  CAST(NULL AS DOUBLE) AS trip_type,    -- not in yellow
  congestion_surcharge,
  trip_time,
  speed_mph,
  airport_fee,
  'yellow' AS taxi_colour                -- added constant column
FROM yellow_taxi_final;


In [0]:
%sql
-- Combine both
CREATE OR REPLACE TABLE combined_taxi AS
SELECT * FROM green_taxi_standard
UNION ALL
SELECT * FROM yellow_taxi_standard;

num_affected_rows,num_inserted_rows


-----
## 5. Combine the table with the location data

In [0]:
%sql
-- Load the location lookup CSV into a table
CREATE OR REPLACE TEMPORARY VIEW taxi_zone_lookup
USING csv
OPTIONS (
  path '/Volumes/workspace/bde/assignment2/taxi_zone_lookup.csv',
  header 'true',
  inferSchema 'true'
);

In [0]:
%sql
-- Create the table with pickup and dropoff locations
CREATE OR REPLACE TABLE combined_taxi_with_zones AS
SELECT
    t.*,
    pu.Borough AS pickup_borough,
    pu.Zone AS pickup_zone,
    do.Borough AS dropoff_borough,
    do.Zone AS dropoff_zone
FROM combined_taxi t
LEFT JOIN taxi_zone_lookup pu
    ON t.PULocationID = pu.LocationID
LEFT JOIN taxi_zone_lookup do
    ON t.DOLocationID = do.LocationID;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Quick sample
SELECT *
FROM combined_taxi_with_zones
LIMIT 20;


VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,trip_time,speed_mph,airport_fee,taxi_colour,pickup_borough,pickup_zone,dropoff_borough,dropoff_zone
1,2014-01-01T05:42:37.000Z,2014-01-01T05:56:09.000Z,2.0,1.7,1.0,Y,75,140,2.0,10.0,1.0,0.5,0.0,0.0,null,0.3,11.8,1.0,null,0.225556,7.5369309617123905,null,green,Manhattan,East Harlem South,Manhattan,Lenox Hill East
1,2014-01-01T06:15:01.000Z,2014-01-01T06:23:52.000Z,1.0,2.4,1.0,Y,75,168,2.0,9.5,0.5,0.5,0.0,0.0,null,0.3,10.8,1.0,null,0.147500,16.271186440677965,null,green,Manhattan,East Harlem South,Bronx,Mott Haven/Port Morris
1,2014-01-01T06:38:17.000Z,2014-01-01T06:47:10.000Z,2.0,1.9,1.0,Y,41,42,2.0,8.5,0.5,0.5,0.0,0.0,null,0.3,9.8,1.0,null,0.148056,12.832982114875453,null,green,Manhattan,Central Harlem,Manhattan,Central Harlem North
1,2014-01-01T07:29:32.000Z,2014-01-01T07:36:43.000Z,1.0,0.7,1.0,Y,75,74,2.0,6.5,0.5,0.5,0.0,0.0,null,0.3,7.8,1.0,null,0.119722,5.846878602094853,null,green,Manhattan,East Harlem South,Manhattan,East Harlem North
1,2014-01-01T07:47:33.000Z,2014-01-01T07:49:39.000Z,1.0,0.9,1.0,Y,75,74,3.0,4.5,0.5,0.5,0.0,0.0,null,0.3,5.8,1.0,null,0.035000,25.71428571428571,null,green,Manhattan,East Harlem South,Manhattan,East Harlem North
1,2014-01-01T09:02:50.000Z,2014-01-01T09:11:32.000Z,1.0,1.6,1.0,Y,152,116,2.0,8.0,0.5,0.5,0.0,0.0,null,0.3,9.3,1.0,null,0.145000,11.03448275862069,null,green,Manhattan,Manhattanville,Manhattan,Hamilton Heights
1,2014-01-01T10:05:49.000Z,2014-01-01T10:16:36.000Z,1.0,2.2,1.0,Y,47,42,3.0,10.0,0.5,0.5,0.0,0.0,null,0.3,11.3,1.0,null,0.179722,12.241127964300421,null,green,Bronx,Claremont/Bathgate,Manhattan,Central Harlem North
1,2014-01-01T10:34:40.000Z,2014-01-01T10:42:24.000Z,1.0,1.3,1.0,Y,42,166,2.0,7.0,0.5,0.5,0.0,0.0,null,0.3,8.3,1.0,null,0.128889,10.086198201553275,null,green,Manhattan,Central Harlem North,Manhattan,Morningside Heights
2,2014-01-10T10:00:24.000Z,2014-01-10T10:15:07.000Z,1.0,2.55,1.0,N,244,169,2.0,11.0,0.5,0.5,0.0,0.0,null,null,12.0,1.0,null,0.245278,10.396366571808315,null,green,Manhattan,Washington Heights South,Bronx,Mount Hope
2,2014-01-10T10:13:36.000Z,2014-01-10T10:19:41.000Z,1.0,0.95,1.0,N,153,220,2.0,6.0,0.0,0.5,0.0,0.0,null,null,6.5,1.0,null,0.101389,9.369852745366853,null,green,Manhattan,Marble Hill,Bronx,Spuyten Duyvil/Kingsbridge


-----
## 6. Save the final cleaned and joined dataset.

In [0]:
%sql
-- Save the table as Delta table
CREATE OR REPLACE TABLE final_taxi_data
USING DELTA
AS
SELECT *
FROM combined_taxi_with_zones;


num_affected_rows,num_inserted_rows


-----
## 7. Count the number of rows of final table

In [0]:
%sql
-- Count the number of records in the final table
SELECT COUNT(*) AS total_count 
FROM final_taxi_data;

total_count
944480149
